# Chapter 04. Training Neural Network

In [25]:
import numpy as np
import matplotlib.pyplot as plt

## 1. 신경망에서의 학습
신경망에서의 학습이란 **train 데이터로부터 weight의 최적값을 구하는 것**을 말한다. 신경망은 다른 방법들과는 다르게 **거의 사람의 개입이 없**다.    
### 1.1. 알고리즘 vs 머신러닝 vs 신경망
<img src="./al_ml_nn.png" width=55%>


* **알고리즘** : 처음부터 끝까지 `사람`이 이미지를 식별하는 알고리즘을 짜는 것
* **머신러닝** : `사람`이 이미지에서 특징을 추출하고 그 특징의 패턴을 `머신러닝`으로 학습하는 것
* **딥러닝** : `신경망`이 주어진 데이터를 학습하고 데이터의 패턴을 찾아내는 것 (=End-to-End Machine Learning)

### 1.2. train 데이터와 test 데이터
머신러닝에서는 데이터를 train 데이터와 test 데이터로 나누어 학습과 검증을 하는 것이 일반적이다. 만약 하나의 데이터만 가지고 학습과 검증을 한다면 올바른 평가가 될 수 없다. 마치 **답을 보고 시험을 보는 것**처럼 그 데이터에 지나치게 최적화된 **오버피팅**이 일어날 수 있기 때문이다.
1. train 데이터를 통해서 최적의 weight값을 찾는다.
2. test 데이터를 이용해 앞에서 만든 모델의 성능을 평가한다.

## 2. 손실 함수 Cost Function
학습을 하는 것은 **최적의 weight값을 찾는 것**이라고 했는데, 어떻게 최적의 weight 값을 찾을 수 있을까? 즉, 어떤 기준으로 좋은 weight인지 아닌지를 알 수 있을까? 이에 대한 지표로 활용하는 것이 바로 **손실 함수**이다. 이 손실함수가 작을수록 최적의 weight값을 찾는 것이라 보면 된다. 

대표적인 손실 함수로는 `평균 오차 제곱(MSE)`와 `교차 엔트로피 오차(CEE)`가 있다.

### 2.1. 평균 제곱 오차 Mean Square Error
평균 제곱 오차(MSE, Mean Square Error)는 예측값과 실제값의 차이를 제곱해 평균을 한 것이라 보면 된다.
$$E = \frac{1}{2}\sum_k(y_k - t_k)^2$$

In [26]:
def mean_squared_error(y, t):
    return 0.5 * np.sum((y-t)**2)

In [27]:
# 실제값
t = np.array([0, 0, 1, 0, 0, 0, 0, 0, 0, 0])

# 예측값1
y1 = np.array([0.1, 0.05, 0.6, 0.0, 0.05, 0.1, 0.0, 0.1, 0.0, 0.0])
print(mean_squared_error(y1, t))

# 예측값2
y2 = np.array([0.1, 0.05, 0.1, 0.0, 0.05, 0.1, 0.0, 0.6, 0.0, 0.0])
print(mean_squared_error(y2, t))

0.09750000000000003
0.5975


### 2.2. 교차 엔트로피 오차 Cross Entropy Error
$$E = -\sum_kt_k\log{y_k}$$

교차 엔트로피 오차(CEE, Cross Entropy Error)는 다음과 같다. $y_t$는 신경망의 출력값(=확률)이고, $t_k$는 레이블값이다. 이때 $t_k$는 정답에 해당하는 것의 요소만 1이고 나머지는 0인 **원-핫 인코딩** 형태이다. 출력값(=확률)이 1에 가까워질수록 $\log{y_k}$의 값이 0에 가까워지므로 오류가 0에 가까워지고, 0에 가까워질수록 해당 값이 무한대로 발산하므로 오류가 증가한다.  

예를 들어, 다른 숫자는 동일하므로 `2`만 봐보자. 나머지 예측값은 **원-핫 인코딩**에 의해 0이 되고 정답 레이블 인덱스인 `2`의 계산값을 보자.
* `[0.1, 0.0, 0.9, 0.0, 0.0, 0.0, 0.1, 0.0, 0.0, 0.0]` : 2에 해당하는 부분만 보면 $-1\log{0.9} = 0.046$이므로 오류가 거의 없다.
* `[0.1, 0.0, 0.3, 0.0, 0.0, 0.0, 0.1, 0.0, 0.0, 0.0]` : 2에 해당하는 부분만 보면 $-1\log{0.3} = 0.52$이므로 오류가 매우 크다.

In [28]:
def cross_entropy_error(y, t):
    # y가 0이 되어버리면 inf가 되므로 그걸 방지하기 위한 상수
    delta = 1e-7        
    return -np.sum(t * np.log(y + delta))

In [29]:
# 실제값
t = np.array([0, 0, 1, 0, 0, 0, 0, 0, 0, 0])

# 예측값1
y1 = np.array([0.1, 0.05, 0.6, 0.0, 0.05, 0.1, 0.0, 0.1, 0.0, 0.0])
print(cross_entropy_error(y1, t))

# 예측값2
y2 = np.array([0.1, 0.05, 0.1, 0.0, 0.05, 0.1, 0.0, 0.6, 0.0, 0.0])
print(cross_entropy_error(y2, t))

0.510825457099338
2.302584092994546


### 2.3. 미니 배치의 필요성
위의 손실 함수는 *하나의 데이터의 손실 함수 값*을 구한 것과 같다. 하지만 우리는 100개 아니 수백만개의 데이터를 가지고 있기에 모든 데이터를 대표할 손실 함수값이 필요하다. 그래서 **모든 데이터의 손실 함수값의 평균**을 사용하면 된다.

$$E = -\frac{1}{N}\sum_n\sum_kt_{nk}\log{y_{nk}}$$

그렇지만 우리가 가지고 있는 데이터의 개수는 너무나도 많다. 모든 데이터의 손실 함수의 값을 구하려면 엄청난 시간이 걸리 것이다. 그래서 우리는 *근사값*을 이용한다. 전체 데이터에서 표본이 될만한 데이터를 랜덤으로 뽑는 것이다. 이러한 방식을 **미니배치 학습**이라고 한다.

In [30]:
import sys, os
sys.path.append(os.pardir)
from dataset.mnist import load_mnist

(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

x_train.shape, t_train.shape

((60000, 784), (60000, 10))

In [31]:
# 무작위로 10개의 index 번호 추출
train_size = x_train.shape[0]    # 60000
batch_size = 10

#0 - 59999 중에 10개를 random으로 추출
batch_mask = np.random.choice(train_size, batch_size)
x_batch = x_train[batch_mask]
t_batch = t_train[batch_mask]

In [32]:
# 원-핫 인코딩을 사용하는 경우
# 미니 배치를 활용한 CEE
def cross_entropy_error(y, t):
    # 1차원 -> 2차원 : 데이터가 하나인 경우
    if y.ndim == 1:
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)
    
    # batch size = 데이터의 개수
    batch_size = y.shape[0]    
    return -np.sum(t * np.log(y + 1e-7)) / batch_size

In [33]:
# 레이블 값 자체를 사용하는 경우
# 미니 배치를 활용한 CEE
def cross_entropy_error_label(y, t):
    # 1차원 -> 2차원 : 데이터가 하나인 경우
    if y.ndim == 1:
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)
    
    # batch size = 데이터의 개수
    batch_size = y.shape[0]    
    # batch size만큼의 y 데이터를 추출 : 예를 들어, 5라면 0부터 4까지의 데이터 
    # 거기의 예측값인 t값에 접근 : 내가 2랑 비교하고 싶다 하면 y의 2번째 column에 접근하면 예측값이 나옴
    return -np.sum(np.log(y[np.arange(batch_size), t] + 1e-7)) / batch_size

In [34]:
t = np.array([0, 0, 1, 0, 0, 0, 0, 0, 0, 0])

# 2일 확률이 가장 높다고 예측
y = np.array([0.1, 0.05, 0.6, 0.0, 0.05, 0.1, 0.0, 0.1, 0.0, 0.0]) 
cross_entropy_error(y,t)

0.510825457099338

### 2.4. 손실 함수의 필요성
신경망 학습 시 **미분**은 매우 중요한 역할을 한다. 학습 시 최적의 weight값과 bias값을 찾기 위해 손실 함수를 계산하고 **이를 가장 작게 하는(거의 0에 가까운) 매개변수** 찾는다. 그래서 매개변수를 살짝만 조정해도 그 효과가 고스란히 손실 함수에 일어나게 된다.
* `정확도`의 경우 매개변수를 살짝 조정하더라도, 손실 함수의 변화가 잘 느껴지지 않는다.
* `계단 함수`의 경우 어느 시점에 값이 0→1(1→0)으로 변화하여, 손실 함수의 변화가 잘 느껴지지 않는다.

## 3. 수치 미분 Numerical Difference
### 3.1. 수치 미분
미분은 특정 순간의 변화량을 말한다. **$\frac{df(x)}{dx}$**에서 **$x$의 변화가 함수 $f(x)$를 얼마나 변화시키느냐**를 의미한다. 하지만 우리는 미분을 직접적으로 구하기 어려우므로 그 근사값을 구한다. 이것을 **수치 미분**이라 한다. 그래서 어느 정도의 오차가 포함이 된다.
* $\lim_{h \rightarrow 0}\frac{f(x+h) - f(x)}{h}$ : $h$가 너무 작으면 반올림 오차 발생(작은 소숫점 생략)
* $\lim_{h \rightarrow 0}\frac{f(x+h) - f(x-h)}{2h}$ : $x$를 중심으로 그 전후의 차이를 구함

In [35]:
def numerical_diff(f, x):
    h = 1e-4
    return (f(x+h) - f(x-h)) / (2*h)

In [36]:
# y = 0.01x^2+0.1x
def func(x):
    return 0.01*x**2+0.1*x

print(numerical_diff(func, 5))     # func의 x값이 5일때의 기울기
print(numerical_diff(func, 10))    # func의 x값이 10일때의 기울기

0.1999999999990898
0.2999999999986347


### 3.2. 편미분
편미분은 **변수가 2개 이상**일 때 하는 미분을 말한다. 이 때 다른 변수들의 값을 **고정**하고 **목표 변수 하나의 기울기**만 구하는 것이다.

In [37]:
# y = x_0^2 + x_1^2
def func2(x):
    return x[0]**2+x[1]**2

# x_0에 대해 편미분 (x_1 = 4.0)
def func2_0(x0):
    return x0*x0 + 4.0**2

# x_1에 대해 편미분 (x_0 = 3.0)
def func2_1(x1):
    return 3.0**2 + x1*x1

print(numerical_diff(func2_0, 3.0))    # x_0에 대한 편미분
print(numerical_diff(func2_1, 4.0))    # x_1에 대한 편미분

6.00000000000378
7.999999999999119


## 4. 기울기
### 4.1.  다변수 함수의 기울기

<img src="gradient.png" width="500">

$x_0 = a, x_1 = b$에서의 기울기는 $(\frac{\partial f}{\partial x_0}, \frac{\partial f}{\partial x_1})$이라는 벡터에 대입을 한 것과 같다.    

이 때 **이 벡터들이 가리키는 방향**이 **손실 함수의 값을 가장 크게 줄일 수 있는 방향**이다.

In [38]:
# 2개 이상의 변수를 갖고 있을 때의 기울기 반환
def numerical_gradient(f, x):
    h = 1e-4
    grad = np.zeros_like(x)    # 변수의 개수만큼의 기울기 벡터 요소 생성
    
    for idx in range(x.size):
        tmp = x[idx]
        
        # f(x+h)
        x[idx] = tmp + h
        fxh1 = f(x)
    
        # f(x-h)
        x[idx] = tmp - h
        fxh2 = f(x)
        
        # 기울기
        grad[idx] = (fxh1 - fxh2) / (2*h)
        
        # 원래 x값 복원
        x[idx] = tmp
        
    return grad

In [39]:
print(numerical_gradient(func2, np.array([3.0, 4.0])))
print(numerical_gradient(func2, np.array([0.0, 2.0])))
print(numerical_gradient(func2, np.array([3.0, 0.0])))

[6. 8.]
[0. 4.]
[6. 0.]


### 4.2. 경사하강법 Gradient Descent

<img src="gradient_descent.png" width=50%>

경사하강법(Gradient Descent)란 **현재 위치에서 기울어진 방향(=기울기가 가리키는 방향)으로 일정 거리만큼 이동하여 손실 함수의 값을 줄이는 법**을 말한다.

$$x_0 = x_0 - \eta\frac{\partial f}{\partial x_0}$$
$$x_1 = x_1 - \eta\frac{\partial f}{\partial x_1}$$

이 때 $\eta$는 학습률(learning rate)를 뜻하며, **매개 변수의 값을 얼마만큼 업데이트할지**를 말한다. 보통 0.01이나 0.001등으로 값을 고정한다. 
* 학습률은 신경망이 알아서 구하는 매개변수가 아닌 사람이 정해야 하는 **하이퍼 파라미터(Hyper Parameter)**이다. 
* 학습률이 너무 작으면 매개 변수 값이 거의 업데이트 되지 않고 끝이나고, 너무 크면 큰 값으로 발산을 한다. 적당히 하자.

In [40]:
# f는 함수, init_x는 초기값, lr은 학습률, step_num은 얼마만큼 업데이트 할지
def gradient_descent(f, init_x, lr=0.01, step_num = 100):
    x = init_x
    
    for i in range(step_num):
        grad = numerical_gradient(f, x)
        x -= lr * grad
        
    return x

In [41]:
init_x = np.array([-3.0, 4.0])
gradient_descent(func2, init_x=init_x, lr=0.1, step_num=100)   # 거의 0에 가깝다.

array([-6.11110793e-10,  8.14814391e-10])

### 4.3. 신경망에서의 기울기
<img src='./nn_gradient.png' width=20%>

신경망 학습 시의 기울기는 **weight 매개변수에 대한 손실 함수의 기울기**를 말한다. 다음은 간단한 신경망을 구현한 `simpleNet` 클래스이다.
* `predict` : weight값으로 출력값 예측
* `loss` : 예측값과 레이블값으로 손실 함수인 CEE를 구해 반환

In [42]:
import sys, os
sys.path.append(os.pardir)  
from common.functions import softmax, cross_entropy_error    
from common.gradient import numerical_gradient       

class simpleNet:
    def __init__(self):
        # weight 매개변수를 정규분포를 가진 random값으로 초기화
        self.W = np.random.randn(2,3)        
    
    # 출력값을 예측
    def predict(self, x):
        return np.dot(x, self.W)

    # 손실 함수의 값을 계산
    def loss(self, x, t):
        z = self.predict(x)
        y = softmax(z)
        
        loss = cross_entropy_error(y, t)
        
        return loss

In [43]:
# 신경망 생성
net = simpleNet()
print('weight :\n', net.W)

weight :
 [[-0.92001158  0.26136941 -1.63415577]
 [-1.29320228 -1.94404359  2.40997752]]


In [44]:
# 출력값 예측
x = np.array([0.6, 0.9])
pred = net.predict(x)
print('출력값 :\n', pred)

출력값 :
 [-1.715889   -1.59281759  1.1884863 ]


In [45]:
# 손실 함수값 계산
t = np.array([0, 0, 1]) 
net.loss(x, t)

0.11041421626183123

In [46]:
# 손실 함수 정의 : 더미 함수
f = lambda w : net.loss(x, t)

# weight에 대한 손실 함수의 기울기
dW = numerical_gradient(f, net.W)
print('기울기에 대한 손실 함수의 기울기 :\n', dW)

기울기에 대한 손실 함수의 기울기 :
 [[ 0.02943369  0.03328848 -0.06272217]
 [ 0.04415053  0.04993272 -0.09408325]]


## 5. 학습 알고리즘 구현

신경망 학습은 다음과 같이 이루어진다. 이 때 데이터를 무작위로 뽑아서 미니 배치를 뽑기 때문에 이 때의 경사하강법을 **확률적 경사하강법(SGD, Stochastic Gradient Descent)**라고 한다.
1. `미니 배치` : train 데이터에서 무작위로 데이터를 뽑는다. (=미니 배치를 선정한다.)
2. `기울기 계산` : weight 매개변수에 대한 미니 배치의 손실 함수의 기울기를 계산한다.
3. `매개변수 업데이트` : 앞에서 계산한 기울기의 방향으로 weight 매개변수를 업데이트 한다.
4. `반복` : 손실 함수의 값이 어느 정도 작아질 때까지 1, 2, 3을 반복한다.

### 5.1. 2층 신경망 클래스 구현

In [47]:
import sys, os
sys.path.append(os.pardir)  
from common.functions import *
from common.gradient import numerical_gradient      

class TwoLayerNet:
    # 초기화 : 입력층 뉴런 수, 은닉층 뉴런 수, 출력층 뉴런 수
    def __init__(self, input_size, hidden_size, output_size, weight_init_std=0.01):
        self.params = {}   # 신경망의 매개변수
        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size)
        self.params['b2'] = np.zeros(output_size)
    
    # 출력값 예측
    def predict(self, x):
        W1, W2 = self.params['W1'], self.params['W2']
        b1, b2 = self.params['b1'], self.params['b2']
    
        a1 = np.dot(x, W1) + b1
        z1 = sigmoid(a1)
        a2 = np.dot(z1, W2) + b2
        y = softmax(a2)
        
        return y
        
    # 손실 함수의 값(=CEE) 계산 
    def loss(self, x, t):
        y = self.predict(x)
        
        return cross_entropy_error(y, t)
    
    # 정확도 계산
    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        t = np.argmax(t, axis=1)
        
        accuracy = np.sum(y == t) / float(x.shape[0])
        return accuracy
        
    # weight에 대한 손실 함수의 기울기를 계산
    def numerical_gradient(self, x, t):
        loss_W = lambda W: self.loss(x, t)
        
        grads = {}   # 손실 함수의 기울기(= 각 매개변수에 대한 편미분 값)
        grads['W1'] = numerical_gradient(loss_W, self.params['W1']) 
        grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])
        
        return grads
    
    # weight에 대한 손실 함수의 기울기를 계산 by 오차역전파법
    # 위의 numerical_gradient 방법은 너무 오래 걸리므로 이것을 일단 사용 
    # 설명은 다음 Chapter에
    def gradient(self, x, t):
        W1, W2 = self.params['W1'], self.params['W2']
        b1, b2 = self.params['b1'], self.params['b2']
        grads = {}
        
        batch_num = x.shape[0]
        
        # forward
        a1 = np.dot(x, W1) + b1
        z1 = sigmoid(a1)
        a2 = np.dot(z1, W2) + b2
        y = softmax(a2)
        
        # backward
        dy = (y - t) / batch_num
        grads['W2'] = np.dot(z1.T, dy)
        grads['b2'] = np.sum(dy, axis=0)
        
        dz1 = np.dot(dy, W2.T)
        da1 = sigmoid_grad(a1) * dz1
        grads['W1'] = np.dot(x.T, da1)
        grads['b1'] = np.sum(da1, axis=0)


        return grads

### 5.2. 미니배치 학습

In [48]:
from dataset.mnist import load_mnist

# train 데이터, test 데이터 분리
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

# train 데이터에서 손실 함수값 기록
train_loss_list = []

# 하이퍼 파라미터 설정
iters_num = 10000    # 반복횟수
train_size = x_train.shape[0]
batch_size = 100     # 배치 크기
learning_rate =0.1   # 학습률

# 신경망 생성
network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

# 반복 학습 -> weight값 업데이트
for i in range(iters_num):
    # 미니 배치 추출
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]
    
    # 기울기 계산
    grad = network.gradient(x_batch, t_batch)
    
    # weight값 업데이트
    for key in ('W1', 'b1', 'W2', 'b2'):
        network.params[key] -= learning_rate * grad[key]
    
    # 학습 경과 기록 : 손실 함수 값 기록
    loss = network.loss(x_batch, t_batch)
    train_loss_list.append(loss)

### 5.3. test 데이터로 모델 성능 평가하기 
이 단계에서는 모델이 train 데이터에 너무 **오버피팅**이 되지 않았는지를 확인한다. 여기서 `epoch`라는 단어가 나오는데 `epoch`란 **학습에서 train 데이터를 모두 소진했을 때의 횟수**를 말한다.
* `train 데이터 10000개, 미니배치 100개` : 1 epoch = 100 iters
* `train 데이터 60000개, 미니배치 100개` : 1 epoch = 600 iters

In [50]:
import sys, os
sys.path.append(os.pardir)  
from dataset.mnist import load_mnist

# train 데이터와 test 데이터로 분리
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

# 2층 신경망 생성
network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

# 하이퍼 파라미터 설정
iters_num = 10000    # 반복 횟수
train_size = x_train.shape[0]
batch_size = 100     # 배치 크기
learning_rate = 0.1  # 학습률

train_loss_list = []    # train 데이터 손실 함수 값 기록
train_acc_list = []     # train 데이터 정확도 기록
test_acc_list = []      # test 데이터 정확도 기록

# 1 epoch 정의
iter_per_epoch = max(train_size / batch_size, 1)

# 반복 학습 -> weight값 업데이트
for i in range(iters_num):
    # 미니배치 추출
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]
    
    # 기울기 계산
    grad = network.gradient(x_batch, t_batch)
    
    # 매개변수 업데이트
    for key in ('W1', 'b1', 'W2', 'b2'):
        network.params[key] -= learning_rate * grad[key]
    
    # train 데이터 손실함수 값 기록
    loss = network.loss(x_batch, t_batch)
    train_loss_list.append(loss)
    
    # 1 epoch당 정확도 기록
    if i % iter_per_epoch == 0:
        train_acc = network.accuracy(x_train, t_train)
        test_acc = network.accuracy(x_test, t_test)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        print("i = " + str(i) + " | train acc, test acc | " + str(train_acc) + ", " + str(test_acc))

i = 0 | train acc, test acc | 0.0993, 0.1032
i = 600 | train acc, test acc | 0.7826166666666666, 0.7872
i = 1200 | train acc, test acc | 0.8769, 0.8806
i = 1800 | train acc, test acc | 0.8971166666666667, 0.9002
i = 2400 | train acc, test acc | 0.9065333333333333, 0.91
i = 3000 | train acc, test acc | 0.91385, 0.9156
i = 3600 | train acc, test acc | 0.9186, 0.9187
i = 4200 | train acc, test acc | 0.9229333333333334, 0.9261
i = 4800 | train acc, test acc | 0.9266, 0.9285
i = 5400 | train acc, test acc | 0.9299833333333334, 0.9309
i = 6000 | train acc, test acc | 0.9338833333333333, 0.9349
i = 6600 | train acc, test acc | 0.93615, 0.9359
i = 7200 | train acc, test acc | 0.9393, 0.94
i = 7800 | train acc, test acc | 0.94045, 0.9394
i = 8400 | train acc, test acc | 0.9441, 0.9429
i = 9000 | train acc, test acc | 0.9456166666666667, 0.9438
i = 9600 | train acc, test acc | 0.94725, 0.9462


오버피팅이 일어난다면, 어느 순간부터 test 데이터의 정확도가 점차 떨어진다. 그러므로 **떨어지기 전의 순간을 포착하는 것**이 중요하다. 이러한 기법을 `early stopping`이라고 하며, 그 외에 오버피팅을 예방하기 위한 방법으로 `weight decay`, `drop out`과 같은 방법이 존재한다.